<a href="https://colab.research.google.com/github/douglasbarbosadelima/Data-Science/blob/master/Deep_bow_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep Bow : classificação de polaridade com base reduzida do yelp (10k)

In [4]:
!pip install tensorflow==2.0.0 

In [0]:
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
import numpy as np
import random

tf.__version__

In [6]:
df=pd.read_csv('yelp_short_10k.csv')
df.head()

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [8]:
df.tail()

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
9995,9995,BFp9V3jzANiEdP5NmIVDRg,OUfAxCaWfFs5Hp_MPstxfA,4HVGgGI5rKvUJcaIV7ub7Q,2,2013-11-17,I've had this place two times now... Just does...,0,0,0
9996,9996,LqxEIponInZxdtwTISZTpg,OUfAxCaWfFs5Hp_MPstxfA,9LAXz_VfATZ0kLdaavkN4g,4,2017-06-18,Cool place. A bit expensive when compared to t...,0,0,0
9997,9997,tRQTBlFcKB65G9Ow3PI1nQ,OUfAxCaWfFs5Hp_MPstxfA,_6hReDyVSr94W2WMO1LJLw,3,2013-09-15,The best vanilla latte I have ever had! The co...,0,0,0
9998,9998,rUFCKDGpuP03miyaPlv99A,OUfAxCaWfFs5Hp_MPstxfA,AghcEVxuULED2z8ibLKTww,3,2013-11-17,I've only had their Moca and it was gross! I'm...,1,0,0
9999,9999,T1RSUw4s8p0QNK1Un32JMQ,Zys2Nzn3tNGDF36BAf1sBA,VLDFjeqpUgWhnVuB_8GuEg,4,2014-02-27,"Went for dinner last night, food was awesome, ...",0,0,0


In [9]:
df['stars'].value_counts()

5    3554
4    2966
3    1716
2     891
1     873
Name: stars, dtype: int64

Para fazermos uma classificação binária entre ótimo ou não, vamos considerar ótimo apenas  4 e o 5 e não ótimo os outros. Isso vai "balancear as classes"

In [0]:
df['classe']=0
filtro=df[df['stars']>=4].index
df.loc[filtro,'classe']=1

In [11]:
df['classe'].value_counts()

1    6520
0    3480
Name: classe, dtype: int64

Vamos tirar "aleatoriamente" 6520-3480 amostras de classe 1 (para simplificar, poderíamos usar critérios mais sofisticados para reduzir a amostra positiva)

In [0]:
index_1=df[df['classe']==1].index
index_1=list(index_1)
index_1
index_1_2=random.sample(index_1,k=3040)
df.drop(index=index_1_2,axis=0,inplace=True)

In [13]:
df['classe'].value_counts()

1    3480
0    3480
Name: classe, dtype: int64

In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
#vect=TfidfVectorizer(stop_words=stop,max_features=10000)
vect=CountVectorizer(stop_words=stop,max_features=15000,ngram_range=(1,2),binary=True)
vect.fit(df['text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=15000, min_df=1,
                ngram_range=(1, 2), preprocessor=None,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [15]:
print(len(vect.vocabulary_))

15000


In [16]:
y=np.array(df['classe'].copy())
vect2 = vect.transform(df['text']).toarray()
X=vect2
X.shape, y.shape
X.shape,y.shape


((6960, 15000), (6960,))

In [0]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(5000,activation='relu',input_shape=(15000,),kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(2000,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(X,y,epochs=3,validation_split=0.3,batch_size=30)

Train on 4872 samples, validate on 2088 samples
Epoch 1/3
4872/4872 [==============================] - 158s 32ms/sample - loss: 2.2083 - accuracy: 0.7537 - val_loss: 1.3644 - val_accuracy: 0.8103
Epoch 2/3
4872/4872 [==============================] - 156s 32ms/sample - loss: 1.1482 - accuracy: 0.8680 - val_loss: 1.3152 - val_accuracy: 0.7989
Epoch 3/3
4872/4872 [==============================] - 156s 32ms/sample - loss: 1.0720 - accuracy: 0.8992 - val_loss: 1.3462 - val_accuracy: 0.7936


In [0]:
amostra="It's an amazing restaurant."
vnova=vect.transform([amostra]).toarray()
model.predict(vnova)

array([[0.6708581]], dtype=float32)

3       Love coming here. Yes the place always needs t...
6       Who would have guess that you would be able to...
8       Not bad!! Love that there is a gluten-free, ve...
11      Server was a little rude.\r\n\r\nOrdered the c...
12      I thought Tidy's Flowers had a great reputatio...
                              ...                        
9994    Awesome location, building, amenities and idea...
9995    I've had this place two times now... Just does...
9996    Cool place. A bit expensive when compared to t...
9997    The best vanilla latte I have ever had! The co...
9998    I've only had their Moca and it was gross! I'm...
Name: text, Length: 6960, dtype: object